In [32]:
import sys
sys.path.append('../lib/')

import pandas as pd
import numpy as np
import geopandas as gpd

import workers
import v_ij
import math
import sweden

from shapely.geometry import Point
import gc

### load Zone

In [2]:
grids = gpd.read_file('../results/grids_vgr_2km_density_deso5.shp')


In [13]:
grids.head()

,density,zone,deso_5,geometry
0,26.055,1,1401A,"POLYGON ((342000.000 6390000.000, 342000.000 6..."
1,26.055,2,1401A,"POLYGON ((342000.000 6396000.000, 342000.000 6..."
2,26.055,3,1401A,"POLYGON ((344000.000 6390000.000, 344000.000 6..."
3,26.055,4,1401A,"POLYGON ((344000.000 6392000.000, 344000.000 6..."
4,26.055,5,1401A,"POLYGON ((344000.000 6394000.000, 344000.000 6..."


### Get distance between zones

In [3]:
# This gives a stacked version
distances = workers.zone_distances(grids)
# This gives a matrix-style dataframe
df_d = distances.unstack(level=1)

Calculating distances between zones...


### Get area and average radius

In [5]:
area = 4 # km

# A=pi*r_average^2

r_average = math.sqrt(area/math.pi) # km

### Get population density and geometry

In [41]:
population_density = dict(zip(grids.zone, grids.density))
geometry = dict(zip(grids.zone, grids.geometry))

### Get $v^{tot}_{ij}$

In [48]:
# parameter = ln(f_max/f_min), f_min = 1/T, f_max = 1 T = 1000
T = 1000
f_max = 1
f_min = 1/T
parameter = math.log(f_max / f_min)

ODM_data = []
for i in grids['zone']:
    for j in grids['zone']:
        element = dict()
        element['origin_grid'] = i
        element['desti_grid'] = j
        if i != j:
            # filter out the error divide by zero
            element['trip_weight'] = v_ij.average_daily_trips(population_density[j], area, r_average, df_d[i][j], parameter) + v_ij.average_daily_trips(population_density[i], area, r_average, df_d[i][j], parameter)
        if i == j:
            element['trip_weight'] = 0
        ODM_data.append(element)
ODM_subset = pd.DataFrame(ODM_data)

### Save ODM_subset between grids

In [49]:
ODM_subset.to_csv("../results/ODM_subset.csv", index=False)

In [54]:
ODM_subset.head()

,origin_grid,desti_grid,trip_weight
0,1,1,0.000000
1,1,2,50.924365
2,1,3,458.319289
3,1,4,229.159644
4,1,5,91.663858


### Test get value

In [70]:
i = 1
j = 1
a = ODM_subset.loc[ (ODM_subset['origin_grid'] == i) & (ODM_subset['desti_grid'] == j)].at[0, 'trip_weight']

0.0


## Load ODM_subset

In [12]:
ODM_subset = pd.read_csv("../results/ODM_subset.csv")

In [71]:
ODM_subset.head()

,origin_grid,desti_grid,trip_weight
0,1,1,0.000000
1,1,2,50.924365
2,1,3,458.319289
3,1,4,229.159644
4,1,5,91.663858


### Load Sweden VGR zone data

In [72]:
zones = gpd.read_file('../dbs/sweden/zones/DeSO/DeSO_2018_v2.shp')
zones.loc[:, 'deso_3'] = zones.loc[:, 'deso'].apply(lambda x: x[:2])
zones_subset = zones.loc[zones['deso_3'] == '14', :]
zones_subset_info = dict(zip(zones_subset['deso'], zones_subset['geometry']))
zone_name = zones_subset['deso']
del zones_subset
gc.collect()

3347

### Get zone level $V_{ij}$

In [79]:
cover = []
for i in zone_name:
    sub_cover = []
    for j in grids.zone:
        point_j = Point(geometry[j].centroid.x, geometry[j].centroid.y)
        if zones_subset_info[i].contains(point_j) == True:
            sub_cover.append(j)
    cover.append(sub_cover)

In [81]:
within = dict(zip(zone_name, cover))

In [90]:
a = 1
a = a + float(ODM_subset.loc[ (ODM_subset['origin_grid'] == 1) & (ODM_subset['desti_grid'] == 2)]['trip_weight'])
print(a)

51.924365410142634


In [91]:
ODM_tmp = []
for i in range(0, len(zone_name)):
    for j in range(i + 1, len(zone_name)):
        element = dict()
        element['origin_main_deso'] = zone_name[i]
        element['desti_main_deso'] = zone_name[j]
        element['trip_weight'] = 0

        if len(within[zone_name[i]]) != 0 and len(within[zone_name[j]]) != 0:
            for ii in within[zone_name[i]]:
                for jj in within[zone_name[j]]:
                    element['trip_weight'] = element['trip_weight'] + float(ODM_subset.loc[ (ODM_subset['origin_grid'] == ii) & (ODM_subset['desti_grid'] == jj)]['trip_weight'])
        ODM_tmp.append(element)
ODM_grid = pd.DataFrame(ODM_tmp)

KeyboardInterrupt: 

In [ ]:
ODM_grid.to_csv('../results/ODM_grid.csv', index=False)

In [ ]:
ODM_grid = pd.read_csv('../results/ODM_grid.csv')

In [ ]:
# Initialise an object for storing the ground-truth data including zones
data_sweden = sweden.GroundTruthLoader()
# Load ground-truth survey data into ODM form
data_sweden.load_odm()

tmp_data = []
for i in range(0, len(zone_name)):
    for j in range(i + 1, len(zone_name)):
        tmp_data.append( float(data_sweden.odm.loc[ (data_sweden.odm['ozone'] == zone_name[i]) & (data_sweden.odm['dzone'] == zone_name[j])]['trip_weight']) )

survey_data = np.array(tmp_data)

In [ ]:
tmp_data1 = []
for i in range(0, len(zone_name)):
    for j in range(i + 1, len(zone_name)):
        tmp_data1.append(float(ODM_grid.loc[ (data_sweden.odm['origin_grid'] == zone_name[i]) & (data_sweden.odm['desti_grid'] == zone_name[j])]['trip_weight']))
model_data = np.array(tmp_data1)

In [ ]:
import matplotlib.pyplot as plt
# delete 0
index = []
length = len(survey_data)
for i in range(0, length):
    if survey_data[i] == 0:
        index.append(i)

survey_data = np.delete(survey_data, index)
model_data = np.delete(model_data, index)

probability_model = np.divide(model_data, model_data.sum())


fig, ax = plt.subplots(figsize=(10, 10))
plt.xscale('log')
plt.yscale('log')
ax.scatter(survey_data, probability_model, facecolor='C0', edgecolor='k')

In [ ]:
model_sum = np.sum(probability_model)
survey_sum = np.sum(survey_data)

numerator = 0
denominator = 0

for i in range(0, len(survey_data)):
    v_model = probability_model[i]
    v_survey = survey_data[i]

    numerator  = numerator + min(v_survey, v_model)

denominator = model_sum + survey_sum

SSI = 2 * numerator / denominator
print(SSI)

In [ ]:
from scipy.stats import spearmanr
from scipy.stats import kendalltau


coef, p = spearmanr(probability_model, survey_data)
alpha = 0.05

print("Spearmans correlation coefficient: %.3f" % coef)
if p > alpha:
    print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
    print('Samples are correlated (reject H0) p=%.3f' % p )


coef, p = kendalltau(probability_model, survey_data)
alpha = 0.05

print("Kendalltau correlation coefficient: %.3f" % coef)
if p > alpha:
    print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
else:
    print('Samples are correlated (reject H0) p=%.3f' % p )